# Chapter 7: Joint Distributions
 
This Jupyter notebook is the Python equivalent of the R code in section 7.7 R, [Introduction to Probability, 1st Edition](https://www.crcpress.com/Introduction-to-Probability/Blitzstein-Hwang/p/book/9781466575578), Blitzstein & Hwang.

----

## Multinomial

The functions for the Multinomial distribution are dmultinom (which is the joint PMF of the Multinomial distribution) and rmultinom (which generates realizations of Multinomial random vectors). The joint CDF of the Multinomial is a pain to work with, so it is not built into R.

To use dmultinom, we have to input the value at which to evaluate the joint PMF, as well as the parameters of the distribution. For example,

In [ ]:
x <- c(2,0,3)
n <- 5 p <- c(1/3,1/3,1/3) 
dmultinom(x,n,p)

returns the probability $P(X1 = 2, X2 = 0, X3 = 3)$, where

\begin{align}
  X = (X1, X2, X3) \sim Mult3(5, (\frac{1}{3}, \frac{1}{3}, \frac{1}{3}))
\end{align}

Of course, $n$ has to equal `sum(x)`; if we attempted to do dmultinom(x,7,p), R would report an error.

For rmultinom, the first input is the number of Multinomial random vectors to generate, and the other inputs are the same. When we typed rmultinom(10,n,p) with n and p as above, R gave us the following matrix: 


    0 2 1 3 2 3 1 2 3 4 
    2 2 2 2 3 0 1 2 0 0 
    3 1 2 0 0 2 3 1 2 1 

Each column of the matrix corresponds to a draw from the Mult3(5,(1/3, 1/3, 1/3)) distribution. In particular, the sum of each column is 5.

----

&copy; Blitzstein, Joseph K.; Hwang, Jessica. Introduction to Probability (Chapman & Hall/CRC Texts in Statistical Science).